# --- Section 1: Introduction to Managed Inference & Agents (MIA) on Heroku ---

## Welcome to the AIEngineer World's Fair Workshop!

This interactive Jupyter Notebook will guide you through the exciting world of Managed Inference and Agents (MIA) on Heroku. We'll explore how Heroku simplifies deploying and interacting with AI models, leveraging powerful Heroku Tools, and understanding the foundational Model Context Protocol (MCP).

**Workshop Agenda:**
1.  **Introduction:** Setting the stage for AI and Agents on Heroku.
2.  **Managed Inference:** Accessing and utilizing AI models through Heroku MIA.
3.  **Heroku Tools:** Empowering agents to take action within your Heroku applications.
4.  **Model Context Protocol (MCP):** The open standard enabling intelligent agent interactions.

---

## Setting up Your Environment (Pre-requisites)

To run this notebook, you'll need:
* A Heroku account.
* A Heroku app with the Heroku Managed Inference and Agents (MIA) add-on attached.
* Environment variables configured for your Heroku MIA endpoint and API key.

```python
# You would typically set these as Heroku Config Vars, but for local testing
# or demonstration purposes, you might set them directly (for the workshop,
# emphasize they should be config vars in a real app).

# import os
#
# # Replace with your actual values (DO NOT COMMIT SENSITIVE KEYS)
# os.environ["INFERENCE_URL"] = "YOUR_HEROKU_MIA_ENDPOINT_URL"
# %env INFERENCE_URL = "YOUR_HEROKU_MIA_ENDPOINT_URL"
# os.environ["INFERENCE_KEY"] = "YOUR_HEROKU_MIA_API_KEY"
# os.environ["INFERENCE_MODEL_ID_CHAT"] = "claude-4-sonnet" # Or other chat model
# os.environ["INFERENCE_MODEL_ID_EMBED"] = "cohere-embed-multilingual" # Or other embedding model
# os.environ["TARGET_APP_NAME"] = "mia-workshop" # The Heroku app your tools will interact with
```

In [ ]:
%env INFERENCE_URL='YOUR_HEROKU_MIA_ENDPOINT_URL'
%env INFERENCE_KEY='YOUR_HEROKU_MIA_API_KEY'
%env INFERENCE_MODEL_ID_CHAT=claude-4-sonnet
%env INFERENCE_MODEL_ID_EMBED=cohere-embed-mutilingual
%env TARGET_APP_NAME=mia-workshop

In [ ]:
# Basic imports
import requests
import json
import os

# Verify environment variables are set (for demonstration purposes)
try:
    INFERENCE_URL = os.environ.get("INFERENCE_URL")
    INFERENCE_KEY = os.environ.get("INFERENCE_KEY")
    INFERENCE_MODEL_ID_CHAT = os.environ.get("INFERENCE_MODEL_ID")
    INFERENCE_MODEL_ID_EMBED = os.environ.get("INFERENCE_MODEL_ID_EMBED")
    TARGET_APP_NAME = os.environ.get("TARGET_APP_NAME") or "mia-workshop"
    print("Environment variables loaded successfully!")
except KeyError as e:
    print(f"Error: Missing environment variable {e}. Please ensure INFERENCE_URL, INFERENCE_KEY, INFERENCE_MODEL_ID_CHAT, INFERENCE_MODEL_ID_EMBED, and TARGET_APP_NAME are set.")
    # Exit or handle gracefully for workshop

# --- Section 2: Managed Inference with Heroku MIA ---
## What is Managed Inference?
Heroku Managed Inference and Agents (MIA) simplifies the deployment and management of state-of-the-art AI models. Instead of handling complex infrastructure, you can access powerful foundational models as a managed service directly from your Heroku applications.

This means:

+ No server management: Heroku handles scaling, patching, and availability.
+ Easy access: Interact with models via simple API calls.
+ Focus on innovation: Concentrate on building intelligent features, not maintaining AI infrastructure.  

Calling a Chat Model
Let's start by making a simple API call to a chat model provisioned via Heroku MIA. We'll ask it to generate some text.

In [ ]:
# Example: Simple Chat Completion

headers = {
    "Authorization": f"Bearer {INFERENCE_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": INFERENCE_MODEL_ID_CHAT,
    "messages": [
        {"role": "user", "content": "Explain the concept of 'Managed Inference' in one sentence."}
    ],
    "max_tokens": 100
}

try:
    response = requests.post(f"{INFERENCE_URL}/v1/chat/completions", headers=headers, json=payload)
    response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
    result = response.json()

    print("Model Response:")
    print(result['choices'][0]['message']['content'])

except requests.exceptions.RequestException as e:
    print(f"Error making API call: {e}")
    if response:
        print(f"Response status: {response.status_code}")
        print(f"Response body: {response.text}")

## Exploring Embedding Models (Optional - if time allows)  

Embedding models convert text into numerical vector representations, which are crucial for tasks like semantic search, recommendation systems, and Retrieval Augmented Generation (RAG).

In [ ]:
# Example: Generating Embeddings (Conceptual)
# This would require an embedding model provisioned in MIA

# payload_embed = {
#     "model": INFERENCE_MODEL_ID_EMBED,
#     "input": ["This is a sentence to embed.", "Another sentence."]
# }
#
# try:
#     response_embed = requests.post(f"{INFERENCE_URL}/v1/embeddings", headers=headers, json=payload_embed)
#     response_embed.raise_for_status()
#     embeddings = response_embed.json()
#     print("\nEmbeddings generated (first few dimensions of first embedding):")
#     print(embeddings['data'][0]['embedding'][:5]) # Print first 5 dimensions
# except requests.exceptions.RequestException as e:
#     print(f"Error generating embeddings: {e}")
#     if response_embed:
#         print(f"Response status: {response_embed.status_code}")
#         print(f"Response body: {response_embed.text}")

# --- Section 3: Heroku Tools: Enabling Agents to Act ---
## The Power of Tools
Large Language Models (LLMs) are incredible at understanding and generating human language, but they don't inherently know how to interact with external systems or perform actions. This is where Heroku Tools come in.

Heroku Tools allow AI agents to:

+ Execute commands: Run shell commands on your Heroku dynos.
+ Query databases: Access and retrieve data from Heroku Postgres.
+ Interact with APIs: Call custom APIs exposed by your Heroku apps.
+ This capability transforms a static LLM into an intelligent agent that can perform actions within your application ecosystem.
## Demo: Running a Command on a Heroku Dyno
Let's demonstrate how an agent can use the dyno_run_command tool to get the current date and time from your Heroku application's dyno.

_Pre-requisite_: Your Heroku app (TARGET_APP_NAME) needs to have the MIA add-on attached and the dyno_run_command tool configured (this is usually done via Heroku CLI or a tool definition in your agent setup).

In [ ]:
# Example: Agent using dyno_run_command to get current time

payload_tool = {
    "model": INFERENCE_MODEL_ID_CHAT,
    "messages": [
        {"role": "user", "content": "What is the current date and time on the server?"}
    ],
    "tools": [
        {
            "type": "heroku_tool",
            "name": "dyno_run_command",
            "runtime_params": {
                "target_app_name": TARGET_APP_NAME,
                "tool_params": {
                    "cmd": "date",
                    "description": "Runs the 'date' command on a one-off dyno to get the current date and time.",
                    "parameters": {"type": "object", "properties": {}, "required": []}
                }
            }
        }
    ]
}

try:
    print(f"Attempting to call Heroku MIA agent endpoint for app: {TARGET_APP_NAME}...")
    response_tool = requests.post(f"{INFERENCE_URL}/v1/agents/heroku", headers=headers, json=payload_tool)
    response_tool.raise_for_status()
    result_tool = response_tool.json()

    print("\nAgent Response (including tool call information):")
    # This structure can vary slightly based on the LLM's tool calling output
    if 'choices' in result_tool and result_tool['choices']:
        message = result_tool['choices'][0]['message']
        if 'tool_calls' in message:
            print("Agent decided to call a tool:")
            for tool_call in message['tool_calls']:
                print(f"  Tool Name: {tool_call['function']['name']}")
                print(f"  Tool Arguments: {tool_call['function']['arguments']}")
        if 'content' in message:
            print("  Agent's final response:")
            print(message['content'])
    else:
        print("No choices found in response or unexpected format.")
        print(result_tool) # Print raw result for debugging

except requests.exceptions.RequestException as e:
    print(f"Error making API call with tool: {e}")
    if response_tool:
        print(f"Response status: {response_tool.status_code}")
        print(f"Response body: {response_tool.text}")

# Demo: Querying a Heroku Postgres Database (Conceptual)
This would involve a similar structure to the dyno_run_command example, but using the postgres_run_query tool. You'd need a Heroku Postgres add-on attached and a follower database.

In [ ]:
# Example: Agent using postgres_run_query (Conceptual)

# payload_db_query = {
#     "model": INFERENCE_MODEL_ID_CHAT,
#     "messages": [
#         {"role": "user", "content": "How many users are in my 'users' table?"}
#     ],
#     "tools": [
#         {
#             "type": "heroku_tool",
#             "name": "postgres_run_query",
#             "runtime_params": {
#                 "target_app_name": TARGET_APP_NAME,
#                 "tool_params": {
#                     "db_attachment": os.environ.get("DATABASE_URL"), # Or specific alias
#                     "query": "SELECT COUNT(*) FROM users;",
#                     "description": "Runs a SQL query on the 'users' table.",
#                     "parameters": {"type": "object", "properties": {}, "required": []}
#                 }
#             }
#         }
#     ]
# }

# ... (similar requests.post and result parsing logic as above)

# --- Section 4: Model Context Protocol (MCP) ---
## What is MCP? The "USB-C for AI"
The Model Context Protocol (MCP) is an open standard, initially introduced by Anthropic, that provides a universal and standardized way for AI models (especially agents) to interact with external data sources, tools, and environments.

Think of it like a USB-C port for AI applications. Instead of requiring bespoke integrations for every tool or data source, MCP defines a common "language" (built on JSON-RPC 2.0) that enables seamless communication.

## Why MCP Matters for Heroku MIA
Heroku's Managed Inference and Agents leverages MCP to provide its powerful agentic capabilities.

+ Standardization: Ensures consistent interaction between AI agents and your applications/tools.
+ Security: Defines a secure framework for context exchange.
+ Scalability: Designed to handle complex interactions across diverse systems.
+ Interoperability: Promotes a richer ecosystem where AI agents can easily integrate with various services.

## How it Works (High-Level Concept)
+ Host Process (Heroku MIA): Manages the lifecycle and security policies for AI agents.
+ Client Instances (AI Model/Agent): Runs within the host, negotiating capabilities and orchestrating messages.
+ MCP Server (Your Heroku App/Tool): Exposes functionalities (like dyno_run_command or postgres_run_query) via MCP.
When an AI agent needs to perform an action (like running a command), it sends an MCP-compliant request to the Heroku MIA host, which then facilitates the secure execution of the tool via the relevant MCP server. The result is returned to the agent, allowing it to incorporate that information into its response or subsequent actions.

This abstraction allows you to focus on what you want your agent to do, rather than the intricate details of how it communicates with your backend systems.

# --- Section 5: Q&A and Next Steps ---
## Key Takeaways
+ Heroku MIA simplifies the deployment and management of AI models.
+ Heroku Tools empower AI agents to perform real-world actions within your applications.
+ Model Context Protocol (MCP) provides the standardized backbone for secure and scalable agent interactions.

## Resources
+ Heroku Managed Inference and Agents: [https://www.heroku.com/ai/managed-inference-and-agents/](https://www.heroku.com/ai/managed-inference-and-agents/)
+ Jupyter Notebooks on Heroku: [https://www.heroku.com/blog/jupyter-notebooks-heroku-persistent-storage/](https://www.heroku.com/blog/jupyter-notebooks-heroku-persistent-storage/)
+ Using Heroku Tools with MIA: [https://devcenter.heroku.com/articles/heroku-inference-tools](https://devcenter.heroku.com/articles/heroku-inference-tools)
+ Model Context Protocol (MCP) Explanation: [https://stytch.com/blog/model-context-protocol-introduction/](https://stytch.com/blog/model-context-protocol-introduction/)
+ This Workshop Notebook: [Link to your GitHub repository (once created)]
